# Py-World
    - Donald Dunagan
    - 811-648-053
    - 11 November 2018
    
    8ish hours

#### Imports

In [1]:
#imports
import pygame
import tcod
import time
import random

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


### Stuff

### Constants

In [2]:
#game sizes
GAME_WIDTH = 480
GAME_HEIGHT =320
CELL_WIDTH = 16
CELL_HEIGHT = 16

#definitions for colors
BLACK_COLOR = (0,0,0)
WHITE_COLOR = (255,255,255,)
GRAY_COLOR = (127,127,127)

#colors for game
BAKGROUND_COLOR = GRAY_COLOR

#map size
MAP_WIDTH = 30
MAP_HEIGHT = 20

#sprites
KNIGHT = pygame.image.load("C:\\Users\\gray\\Desktop\\AI\\Game\\PyWorld\\character_images\\knight.png")
ROGUE = pygame.image.load("C:\\Users\\gray\\Desktop\\AI\\Game\\PyWorld\\character_images\\rogue.png")
ORC = pygame.image.load("C:\\Users\\gray\\Desktop\\AI\\Game\\PyWorld\\character_images\\orc.png")

#map tiles
GRASSY_TILE = pygame.image.load("C:\\Users\\gray\\Desktop\\AI\\Game\\PyWorld\\tile_images\\grassy_tile.png")
TREE = pygame.image.load("C:\\Users\\gray\\Desktop\\AI\\Game\\PyWorld\\tile_images\\tree.png")

#FPS limit
GAME_FPS = 60

#message defaults
NUM_MESSAGES = 4

### Structures

In [3]:
class Struc_Tile:
    def __init__(self, blocks_path):
        self.blocks_path = blocks_path

### Objects

In [4]:
class obj_Actor:
    def __init__(self, x, y, name_object, sprite, creature=None, ai =None):
        #map x address, not pixel address
        self.x = x
        #map y address, not pixel address
        self.y = y
        self.sprite = sprite
        
        self.creature =creature
        if creature:
            creature.owner = self
        
        self.ai = ai
        if ai:
            ai.owner = self
    
    def draw(self):
        #draw the sprite
        MAIN_SURFACE.blit(self.sprite, (self.x*CELL_WIDTH,self.y*CELL_HEIGHT)) 
        


### Components

In [5]:
#components make up larger objects

class com_Creature:
    '''
    creatures have health.
    creatures can damage the heros.
    craetures can die.
    '''
    def __init__(self,name_instance, hp = 10, death_function=None ):
        self.name_instance = name_instance
        self.max_hp = hp
        self.hp = hp
        self.death_function = death_function
        
        
    def move(self, deltaX, deltaY):
        #boolean for whether a tile is a tree or not
        tile_is_tree = (GAME_MAP[self.owner.x +deltaX][self.owner.y+deltaY].blocks_path == True)

        target = map_check_for_creature(self.owner.x +deltaX,self.owner.y+deltaY, self.owner)

        if target:
            self.attack(target, 3)

        if not tile_is_tree and target is None:        
            self.owner.x += deltaX
            self.owner.y += deltaY  
            
    def attack(self, target, damage):
        game_message(self.name_instance + " attacks " +target.creature.name_instance+ " for "+str(damage)+" damage!",WHITE_COLOR)
        target.creature.take_damage(damage)
        
    def take_damage(self, damage_amount):
        self.hp -= damage_amount
        game_message (self.name_instance+"'s health is "+ str(self.hp)+"/"+str(self.max_hp),WHITE_COLOR)
        
        if self.hp <=0:
            if self.death_function is not None:
                self.death_function(self.owner)
    
    
#class com_Item:
    
#class com_Container:

### AI

In [6]:
class ai_Test:
    '''
    Once per turn, do something
    '''
    def take_turn(self):
        
        #slow things down
        #time.sleep(.5)
        
        #randomely move left, right, up, down, or not at all
        randint = random.randint(1,5)
        #no move
        if (randint ==1):
            self.owner.creature.move(0,0)
        #move left
        if randint == 2:
            self.owner.creature.move(-1,0)
        #move right
        if randint==3:
            self.owner.creature.move(1,0)
        #move down
        if randint == 4:
            self.owner.creature.move(0,1)
        #move up
        if randint==5:
            self.owner.creature.move(0,-1)        
        
        

### Death Function

In [7]:
def death_monster(monster):
    '''
    monsters stop moving when dead
    '''
    game_message(monster.creature.name_instance+" is dead!",WHITE_COLOR)
    monster.creature = None
    monster.ai = None
    
    return

### Map

In [8]:
def Create_Map():
    
    #map has a matrix representation
    new_map = [[Struc_Tile(False) for y in range(0,MAP_HEIGHT)]for x in range(0,MAP_WIDTH)]
    
    #Set boundaries at edge of world so that agents cannot leave
    for x in range (MAP_WIDTH):     
        new_map[x][0].blocks_path = True
        new_map[x][MAP_HEIGHT-1].blocks_path = True
    for y in range(MAP_HEIGHT):
        new_map[0][y].blocks_path = True
        new_map[MAP_WIDTH-1][y].blocks_path = True    
        
    #generate some trees in middle of map
    new_map[7][9].blocks_path = True
    new_map[11][4].blocks_path = True
    new_map[13][17].blocks_path = True
    new_map[19][2].blocks_path = True
    new_map[16][12].blocks_path = True
    new_map[4][12].blocks_path = True
    new_map[3][16].blocks_path = True
    new_map[24][8].blocks_path = True
    new_map[26][15].blocks_path = True
    
    return new_map

def map_check_for_creature(x, y, exclude_object = None):
    
    target = None
    
    #if targettable creature is in location, set it to target
    if exclude_object:
        for obj in GAME_OBJECTS:
            if (obj is not exclude_object and 
                obj.x == x and 
                obj.y == y and 
                obj.creature):
                target = obj
            if target:
                return target
    #target any object at location
    else:
        for obj in GAME_OBJECTS:
            if (obj.x == x and 
                obj.y == y and 
                obj.creature):
                target = obj
            if target:
                return target

### Drawing

In [9]:
def Draw_Game():
    
    global MAIN_SURFACE, GAME_MAP, knight, orc1, orc2, orc3, GAME_OBJECTS
    
    #clear the surface
    MAIN_SURFACE.fill(BAKGROUND_COLOR)
    
    #draw the map
    Draw_Map(GAME_MAP)
    
    #draw all of the game objects
    for obj in GAME_OBJECTS:
        obj.draw()
    
    #draw debug messages
    draw_debug()
    
    #messages
    draw_messages()
    
    #update display
    #puts all updates on display
    pygame.display.flip()
    
    
    return

def Draw_Map(map):
    
    for x in range(0,MAP_WIDTH):
        for y in range(0,MAP_HEIGHT):
            if map [x][y].blocks_path == True:
                #draw tree
                MAIN_SURFACE.blit(TREE,(x*CELL_WIDTH,y*CELL_HEIGHT))
            else:
                #draw grass
                MAIN_SURFACE.blit(GRASSY_TILE,(x*CELL_WIDTH,y*CELL_HEIGHT))
    
    return

def draw_debug():
    draw_text(MAIN_SURFACE, "fps:"+str(int(CLOCK.get_fps())), (0,0), WHITE_COLOR, BLACK_COLOR)
    
def draw_messages():
    if len(GAME_MESSAGES) <= NUM_MESSAGES: 
        to_draw = GAME_MESSAGES
    else:
        to_draw = GAME_MESSAGES[-NUM_MESSAGES:]
               
    text_height = helper_text_height(FONT_MESSAGE_TEXT)
    
    start_y = MAP_HEIGHT*CELL_HEIGHT - (NUM_MESSAGES * text_height)
    
    i = 0
    for message, color in to_draw:
        draw_text(MAIN_SURFACE, message, (0,start_y + (i*text_height)), WHITE_COLOR, BLACK_COLOR)
        i +=1

def draw_text(display_surface, text_to_display, T_coords, text_color, back_color = None):
    '''
    takes text and displays on the given surface
    '''
    text_surf, text_rect = helper_text_objects(text_to_display, text_color, back_color)
    
    text_rect.topleft = T_coords
    
    display_surface.blit(text_surf, text_rect)


### Helpers

In [10]:
def helper_text_objects(incoming_text, incoming_color, incoming_bg):
    #renders given text in font and returns rendered text and the surface rectangle    
    if (incoming_bg != None):
        Text_surface = FONT_DEBUG_MESSAGE.render(incoming_text, False, incoming_color, incoming_bg)
    else:
        Text_surface = FONT_DEBUG_MESSAGE.render(incoming_text, False, incoming_color)                        
        
        
    return Text_surface, Text_surface.get_rect()

def helper_text_height(font):
    font_object = font.render('a', False, (0,0,0))
    font_rect = font_object.get_rect()
    
    return font_rect.height

### Game Message

In [11]:
def game_message(game_msg, msg_color):
    
    GAME_MESSAGES.append((game_msg, msg_color))    
    return

### Initialize the Game

In [12]:
def Initialize_Game():
    '''
    initializes game window and pygame    
    '''
    
    #global vars for main surface and for game map
    global MAIN_SURFACE, GAME_MAP, knight, orc1, orc2, orc3, GAME_OBJECTS, FONT_DEBUG_MESSAGE, CLOCK, FONT_MESSAGE_TEXT, GAME_MESSAGES
    
    CLOCK = pygame.time.Clock()
      
    #initialize pygame
    pygame.init()
    
    #font
    #https://www.dafont.com/joystix.font
    FONT_DEBUG_MESSAGE = pygame.font.Font("C:\\Users\\gray\\Desktop\\AI\\Game\\PyWorld\\font\\joystix_monospace.ttf",12)
    FONT_MESSAGE_TEXT = pygame.font.Font("C:\\Users\\gray\\Desktop\\AI\\Game\\PyWorld\\font\\joystix_monospace.ttf",12)
    
    
    #initialize the main surface as GAME_WIDTH x GAME_HEIGHT pixels
    MAIN_SURFACE = pygame.display.set_mode((MAP_WIDTH*CELL_WIDTH,MAP_HEIGHT*CELL_HEIGHT))

    #generate the game map
    GAME_MAP = Create_Map()
    
    #game messages
    GAME_MESSAGES = []

    
    
    #generate knight object
    creature_com1 = com_Creature("greg")
    #ai_com1 =ai_Test()
    knight = obj_Actor(1,1, "knight",KNIGHT, 
                       creature=creature_com1)
    
    #generate orc1
    creature_com2 = com_Creature("jackie",death_function = death_monster)
    ai_com2 = ai_Test()
    orc1 = obj_Actor(27,3,"orc1",ORC, 
                     creature = creature_com2, 
                     ai = ai_com2)
    '''
    #generate orc2
    creature_com3 = com_Creature("steve",death_function = death_monster)
    ai_com3 = ai_Test()
    orc2 = obj_Actor(8,17,"orc2",ORC, 
                     creature = creature_com3, 
                     ai = ai_com3)
    
    #generate orc3
    creature_com4 = com_Creature("geoff",death_function = death_monster)
    ai_com4 = ai_Test()
    orc3 = obj_Actor(15,10,"orc3",ORC, 
                     creature = creature_com4, 
                     ai = ai_com4)
    '''


    GAME_OBJECTS = [knight, orc1]

    
    
    return

### Game Loop

In [13]:
def Game_Loop():
    '''
    runs game loop until the game is over  
    '''
    
    #variable for terminating the game loop
    game_over = False
    
    #player action definition
    player_action = "no-action"
    
    while not(game_over):

        #time.sleep(5)
        
        '''
        #get input
        #list of input (keyboard and mouse) for every frame that is run
        event_list = pygame.event.get()

        #process input
        for event in event_list:

            #event = 'click X button' -> quit game
            if event.type == pygame.QUIT:
                game_over=True 
                
                
        for obj in GAME_OBJECTS:
            
            if obj.ai:
                
                obj.ai.take_turn()
        '''
        
        
        #handle player input
        player_action = Game_Handle_Keys()
        
        if player_action == "QUIT":
            game_over = True
        elif player_action != "no-action":
            for obj in GAME_OBJECTS:
                if obj.ai:
                    obj.ai.take_turn()
        
                
        #draw the game
        Draw_Game()

        CLOCK.tick(GAME_FPS)

        
        
        
    #TODO quit the game and cleanup environment
    pygame.quit()
    #exit()
      
    return


### Game Handle Keys

In [14]:
def Game_Handle_Keys():
    
    #get input
    #list of input (keyboard and mouse) for every frame that is run
    event_list = pygame.event.get()
    
    #process input
    for event in event_list:

        #event = 'click X button' -> quit game
        if event.type == pygame.QUIT:
            return "QUIT"
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                knight.creature.move(0,-1)
                return "player-moved"
            if event.key == pygame.K_DOWN:
                knight.creature.move(0,1)
                return "player-moved"
            if event.key == pygame.K_LEFT:
                knight.creature.move(-1,0)
                return "player-moved"
            if event.key == pygame.K_RIGHT:
                knight.creature.move(1,0)
                return "player-moved"
    return "no-action"

### Stuff

### "main"

In [15]:
Initialize_Game()
Game_Loop()